detectron2 using cocolike structure training mask rcnn for 5000 iterations.

In [ ]:
!pip install -q -U pyyaml==5.1 'pycocotools>=2.0.1'
!pip install -q -U torch==1.6.0+cu102 torchvision==0.7.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q -U pycocotools --use-feature=2020-resolver

import torch, torchvision
print(torch.__version__, torch.cuda.is_available(), torch.version.cuda)

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.6/index.html

In [ ]:
import os
import copy
import cv2
import json
import pycocotools
import detectron2
import random 

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

assert torch.__version__.startswith("1.6")
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances 
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [ ]:
register_coco_instances("my_dataset_train",{},"../input/ut-airbus-preprocess/test_annotations.json","../input/airbus-ship-detection/train_v2/") #TRAIN annotations got mixed up -.-
register_coco_instances("my_dataset_val",{},"../input/ut-airbus-preprocess/train_annotations.json","../input/airbus-ship-detection/train_v2/")

my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

for d in random.sample(dataset_dicts, 6): #Random 6 pictures from the dataset, can be only sea or ship too... 
    img = cv2.imread(d['file_name'])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.figure(figsize = (10,10))
    plt.imshow(vis.get_image()[:, :, ::-1])

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
#cfg.TEST.EVAL_PERIOD = 500
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train() #Trainer will throw out non-annotated pictures. 

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Should build valuator inside the training. So it could be evaluated from there. 

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir="../output/") #Should bne val but ffs.  
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator)) #https://medium.com/@yanfengliux/the-confusing-metrics-of-ap-and-map-for-object-detection-3113ba0386ef

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer

checkpointer = DetectionCheckpointer(trainer.model, save_dir="./")
checkpointer.save("model_mask_resnet101_rcnn")  # save to save_dir

In [ ]:
#https://www.kaggle.com/ptaneja/detectron2-notebook
import glob

def create_test_datatset():    
    img_dir = '../input/airbus-ship-detection/test_v2/'
    dataset_dicts = []
    
    for img_path in glob.glob(img_dir + '*.jpg'):
        record = {}
        file_path = img_path
        image_id = img_path.split('/')[-1].split('.')[0]
        record['file_name'] = file_path
        record['image_id'] = image_id
        dataset_dicts.append(record)
    return dataset_dicts

test_dataset = create_test_datatset()
img_ids = []
pred_string = []

DatasetCatalog.register("submit_test1", create_test_datatset)
od_dataset = MetadataCatalog.get("submit_test1")

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("submit_test1", )
predictor = DefaultPredictor(cfg)

In [ ]:
for test_data in test_dataset[0:30]:
    image_id = test_data['file_name'].split('/')[-1].split('.')[0]
    img = plt.imread(test_data['file_name'])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1], metadata=od_dataset, scale=0.3,)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu") )
    plt.figure(figsize=(25, 15))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
def PixelsToRLenc(pixels ,order='F',format=True):
    """
    Based off code by https://www.kaggle.com/alexlzzz
    pixels is a list of absolute pixel values which need to be converted. (1-243600)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not
    
    returns run length as an array or string (if format is True)
    """
    
    # Initialse empty array
    bytes = []
    for _ in range(0, 243600):
        bytes.append(0)
    
    # Place values from input list into the array
    for x in pixels:
        p = x - 1
        bytes[p] = 1
    
    runs = [] ## list of run lengths
    r = 0     ## the current run length
    pos = 1   ## count starts from 1 per WK
    for c in bytes:
        if ( c == 0 ):
            if r != 0:
                runs.append((pos, r))
                pos+=r
                r=0
            pos+=1
        else:
            r+=1

    #if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''
    
        for rr in runs:
            z+='{} {} '.format(rr[0],rr[1])
        return z[:-1]
    else:
        return runs


img_ids = []
pred_string = []
i = 0


# Put in how long it takes. 
#for test_data in test_dataset[24:25]:
#    i = i + 1
#    image_id = test_data['file_name'].split('/')[-1].split('.')[0]
#    img = plt.imread(test_data['file_name'])
#    outputs = predictor(img)
#    preds = []
#    for box,score in zip(outputs['instances'].get_fields()['pred_boxes'], outputs['instances'].get_fields()['scores']):
#        bbox = []
#        for idx in range(4):
#            bbox.append(box.data[idx].item())
#        preds.append("{} {} {} {}".format(int(bbox[0]),int(bbox[1]), int(bbox[2]), int(bbox[3])))
#        print(preds)
#    if(i%1000 == 0):
#        print(i,len(test_dataset))
#    pred_string.append(" ".join(preds))
#    img_ids.append(test_data['file_name'])
    
    

In [ ]:
#sub={"ImageId":img_ids, "EncodedPixels":PixelsToRLenc(pred_string)}
#sub=pd.DataFrame(sub)

In [ ]:
#sub.to_csv('/kaggle/working/submission.csv',index=False)

In [ ]:
#!kaggle competitions submit -c airbus-ship-detection -f submission.csv -m "Test1"